# API Client
The following tutorial demonstrates how to access the PKDB API with python.

For more information and installation information see:  
https://github.com/matthiaskoenig/pkdb

In [1]:
URL_PKDB = "http://0.0.0.0:8000/"

In [2]:
# import libraries
import coreapi
import requests
import json
from pprint import pprint
from requests.auth import HTTPBasicAuth

In [3]:
#response = requests.get('http://0.0.0.0:8000/', auth=HTTPBasicAuth('janek', 'test'))

In [4]:
#r = requests.post("http://0.0.0.0:8000/api-auth/login/?next=/api/v1/", data={'username': 'janek', 'password': 'test'})
#print(r.status_code, r.reason)
#r.json()["token"]

## Create client & API endpoint

In [5]:
#auth = coreapi.auth.BasicAuthentication(
#username="janek",
#password="test", )
client = coreapi.Client()

In [66]:
# get the api scema
document = client.get(URL_PKDB)
print(document.title)
print(document.url)

PkBD API
http://0.0.0.0:8000/


In [67]:
# overview available methods
print(document)

<PkBD API "http://0.0.0.0:8000/">
    authors: {
        list([page], [first_name], [last_name], [search])
        create([first_name], [last_name])
        read(id, [first_name], [last_name], [search])
        update(id, [first_name], [last_name], [first_name], [last_name], [search])
        partial_update(id, [first_name], [last_name], [first_name], [last_name], [search])
        delete(id, [first_name], [last_name], [search])
    }
    characteristic_values: {
        list([page], [choice], [count], [category], [search])
        create(count, category, [choice], [value], [mean], [median], [min], [max], [sd], [se], [cv], [unit], [group])
        read(id, [choice], [count], [category], [search])
        update(id, count, category, [choice], [value], [mean], [median], [min], [max], [sd], [se], [cv], [unit], [group], [choice], [count], [category], [search])
        partial_update(id, [choice], [count], [value], [mean], [median], [min], [max], [sd], [se], [cv], [unit], [category], [group

## Example queries
Show all authors

In [8]:
data = client.action(document,["authors", "list"])
print(json.dumps(data, indent=2))

{
  "count": 0,
  "next": null,
  "previous": null,
  "results": []
}


Create author

In [32]:
data = client.action(document,["authors", "create"], params={'first_name':'Lucek', 'last_name':'Grzegorzewski'})
print(data)

OrderedDict([('id', 1), ('first_name', 'Lucek'), ('last_name', 'Grzegorzewski')])


In [10]:
# Filter for authors first name = Lucek
data = client.action(document, ["authors", "list"], params={"first_name": "Lucek"} )
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


In [26]:
# Search for author
data = client.action(document, ["authors", "list"], params={"search": "Gr"} )
print(json.dumps(data, indent=2))
import numpy as np

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


In [51]:
# Create study
reference_dict = {
    "pmid": 12087345,
    #"sid": "Haller2002",
    "date": "2002-07-16",
    "journal": "Clinical pharmacology and therapeutics",
    "title": "Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.",
    "abstract": "Botanical stimgle dose.",
    "authors": [
        {
            "first_name": "Christine A Bansakodsfjkopasdfkpodfkanodi asdin",
            "last_name": "Haller"
        },
        {
            "first_name": "Peyton",
            "last_name": "Jacob"
        },
        {
            "first_name": "Neal L",
            "last_name": "Benowitz"
        }
    ],
    "doi": "",
        
        }
    


client.action(document, ["references", "create"], params=reference_dict)

ErrorMessage: <Error: 400 Bad Request>
    authors: [
        {
            first_name: [
                "Ensure this field has no more than 30 characters."
            ]
        },
        {},
        {}
    ]

In [79]:
# Create study
study_dict = {
    "sid":"test",
    "reference":12087345,
    "groups": [
        {
            "count": 14,
            "sid": "Seng2009-S_NCNS",
            "description": "59 healthy adult Asian volunteers were included. In what follows, a non-caffeine consumer (NCCS) refers to an individual who consumed beverages\n(coffee or tea) containing <=200 mg of caffeine per day, and a regular caffeine consumer to one who consumed more >200 mg. A cup (150 mL) of coffee or tea is equivalent to approximately a caffeine dose of 70 and 25 mg respectively (9). Additionally, a non-smoker refers to an individual who did not smoke for at least 6 months prior to dosing. The study population consisted of 14 non-caffeine consumers, non-smokers (NCNS) [age (mean \u00b1 SD) 21 \u00b1 2 years, body weight 62 \u00b1 9 kg), 15 caffeine consumers, non smokers (CNS) (age 24 \u00b1 4 years, body weight 69 \u00b1 12 kg), and 30 caffeine consumer, smokers (CS) (actual consumption 10 \u00b1 4 cigarettes daily, age 23 \u00b1 5 years, body weight 71 \u00b1 19 kg).",
            "characteristic_values": [
                {
                    "count": 14,
                    "category": "sex",
                    "choice": "M"
                },
                {
                    "count": 14,
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "count": 14,
                    "mean": 62.0,
                    "sd": 9.0,
                    "unit": "kg"
                },
                {
                    "count": 14,
                    "category": "ethnicity",
                    "choice": "Asian"
                },
        
            ]
        },
        {
            "count": 15,
            "sid": "Seng2009-S_CNS",
            "description": "59 healthy adult Asian volunteers were included. In what follows, a non-caffeine consumer (NCCS) refers to an individual who consumed beverages\n(coffee or tea) containing <=200 mg of caffeine per day, and a regular caffeine consumer to one who consumed more >200 mg. A cup (150 mL) of coffee or tea is equivalent to approximately a caffeine dose of 70 and 25 mg respectively (9). Additionally, a non-smoker refers to an individual who did not smoke for at least 6 months prior to dosing. The study population consisted of 14 non-caffeine consumers, non-smokers (NCNS) [age (mean \u00b1 SD) 21 \u00b1 2 years, body weight 62 \u00b1 9 kg), 15 caffeine consumers, non smokers (CNS) (age 24 \u00b1 4 years, body weight 69 \u00b1 12 kg), and 30 caffeine consumer, smokers (CS) (actual consumption 10 \u00b1 4 cigarettes daily, age 23 \u00b1 5 years, body weight 71 \u00b1 19 kg).",
            "characteristic_values": [
                {
                    "count": 15,
                    "category": "sex",
                    "choice": "M"
                },
                {
                    "count": 15,
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "count": 15,
                    "mean": 69.0,
                    "sd": 12.0,

                    "unit": "kg"
                },
            ]
        },
        {
            "count": 30,
            "sid": "Seng2009-S_CS",
            "description": "59 healthy adult Asian volunteers were included. In what follows, a non-caffeine consumer (NCCS) refers to an individual who consumed beverages\n(coffee or tea) containing <=200 mg of caffeine per day, and a regular caffeine consumer to one who consumed more >200 mg. A cup (150 mL) of coffee or tea is equivalent to approximately a caffeine dose of 70 and 25 mg respectively (9). Additionally, a non-smoker refers to an individual who did not smoke for at least 6 months prior to dosing. The study population consisted of 14 non-caffeine consumers, non-smokers (NCNS) [age (mean \u00b1 SD) 21 \u00b1 2 years, body weight 62 \u00b1 9 kg), 15 caffeine consumers, non smokers (CNS) (age 24 \u00b1 4 years, body weight 69 \u00b1 12 kg), and 30 caffeine consumer, smokers (CS) (actual consumption 10 \u00b1 4 cigarettes daily, age 23 \u00b1 5 years, body weight 71 \u00b1 19 kg).",
            "characteristic_values": [
                {
                    "count": 30,
                    "category": "sex",
                    "choice": "M"
                },
                {
                    "count": 30,
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "count": 30,
                    "mean": 71.0,
                    "unit": "kg"
                },
                {
                    "count": 30,
                    "category": "ethnicity",
                    "choice": "Asian"
                },
                {
                    "count": 30,
                    "category": "alcohol_abstinence",
                    "choice": "N"
                },

            ]
        }
    ],
        
        }
    


In [80]:
client.action(document, ["studies", "create"], params=study_dict)

ErrorMessage: <Error: 400 Bad Request>
    sid: [
        "study with this sid already exists."
    ]

In [116]:
import json
with open('/home/janekg89/Develop/Pycharm_Projects/pkdb/pkdb_app/data/Master/Studies/Kaplan1997/data.json') as f:
    data = json.load(f)


In [20]:
client.action(document, ["references", "list"])

OrderedDict([('count', 2),
             ('next', None),
             ('previous', None),
             ('results',
              [OrderedDict([('pmid', '12087345'),
                            ('sid', 'Haller2002'),
                            ('doi', ''),
                            ('title',
                             'Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.'),
                            ('abstract', 'Botanical stimgle dose.'),
                            ('journal',
                             'Clinical pharmacology and therapeutics'),
                            ('date', '2002-07-16'),
                            ('authors', []),
                            ('pdf', None)]),
               OrderedDict([('pmid', '12087345'),
                            ('sid', '12087345'),
                            ('doi', ''),
                            ('title',
                             'Pharmacology of ephedra alkaloids and caffeine after 

In [117]:
client.action(document,["references", "create"], params=data)

ParameterError: {'groups': 'This parameter is required.'}

In [62]:
charaVal = {
                    "category": "caffeine_amount",
                    "count": 8,
                    "mean": None,
                    "median": None,
                    "min": None,
                    "max": None,
                    "sd": None,
                    "se": None,
                    "cv": None,
                    "unit": None
                }
client.action(document,["characteristic_values", "create"], params=charaVal)

ErrorMessage: <Error: 400 Bad Request>
    group: [
        "This field is required."
    ]

In [35]:
reference_dict

{'pmid': 12087345,
 'sid': 'Haller2002',
 'date': '2002-07-16',
 'journal': 'Clinical pharmacology and therapeutics',
 'title': 'Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.',
 'abstract': 'Botanical stimulants have disposition characteristics similar to their pharmaceutical counterparts, and they can produce significant cardiovascular responses after a single dose.',
 'authors': [{'first_name': 'Christine A', 'last_name': 'Haller'},
  {'first_name': 'Peyton', 'last_name': 'Jacob'},
  {'first_name': 'Neal L', 'last_name': 'Benowitz'}],
 'doi': '',
 'groups': [{'count': 8,
   'sid': 'Haller2002-S1',
   'description': 'Eight healthy volunteers (5 women and 3 men) from 25 to 38 years old participated in the study. Eligibility for the study was determined on the basis of medical history, brief physical examination, and screening laboratory tests that included complete blood count, serum chemistry values to assess liver and renal function, urine t

In [48]:
path ="/home/janekg89/Develop/Pycharm_Projects/pkdb/pkdb_app/data/caffeine/Subjects.tsv"

In [53]:
def subjects_load(json_reference):
    data_pd = pd.read_csv(path,delimiter='\t')
    this_data = data_pd[data_pd["study"] == json_reference["json"]["sid"]]
    json = {**json_reference["json"]}
    json["groups"] = []
    print(len(this_data))
    for name, row in this_data.iterrows():
        yield {"json": json, "group": row.to_dict()}# "reference_path": json_reference["reference_path"]}

In [54]:
data = {"json" : study_dict }
this = subjects_load(data)

In [55]:
next(this)

2


{'json': {'pmid': 16198659,
  'sid': 'Granfors2005',
  'date': '2005-11-02',
  'journal': 'Clinical pharmacology and therapeutics',
  'title': 'Oral contraceptives containing ethinyl estradiol and gestodene markedly increase plasma concentrations and effects of tizanidine by inhibiting cytochrome P450 1A2.',
  'abstract': 'OCs containing ethinyl estradiol and gestodene increase, to a clinically significant extent, the plasma concentrations and effects of tizanidine, probably mainly by inhibiting its CYP1A2-mediated presystemic metabolism. Care should be exercised when tizanidine is prescribed to OC users.',
  'groups': [],
  'authors': [{'first_name': 'Marika T', 'last_name': 'Granfors'},
   {'first_name': 'Janne T', 'last_name': 'Backman'},
   {'first_name': 'Jouko', 'last_name': 'Laitila'},
   {'first_name': 'Pertti J', 'last_name': 'Neuvonen'}],
  'doi': ''},
 'group': {'study': 'Granfors2005',
  'subjects': 'S1',
  'keywords': 'control',
  'species': 'homo sapiens',
  'strain': '-'

In [60]:
"33.7".replace('.','',1).isdigit()

True